# Blue Sky Run Engine

Contents:

* The Run Engine processes messages
* There is two-way communication between the message generator and the Run Engine
* Control timing with 'sleep' and 'wait'
* Runs can be aborted
* Data can be consumed by Metadatastore and User Functions

In [1]:
%run bs.py

motor = Mover('motor', ['pos'])
det = SynGauss('sg', motor, 'pos', center=0, Imax=1, sigma=1)

:0: FutureWarning: IPython widgets are experimental and may change in the future.


## The Run Engine processes messages

A message has four parts: a command string, an object, a tuple of positional arguments, and a dictionary of keyword arguments.

In [3]:
Msg('set', motor, {'pos': 5})

set: (mover: motor), ({'pos': 5},), {}

In [4]:
Msg('trigger', motor)

trigger: (mover: motor), (), {}

In [5]:
Msg('read', motor)

read: (mover: motor), (), {}

In [2]:
RE = RunEngine()

In [7]:
def simple_scan(motor):
    "Set, trigger, read"
    yield Msg('set', motor, {'pos': 5})
    yield Msg('trigger', motor)
    yield Msg('read', motor)
    
RE.run(simple_scan(motor))

Emitted RunStart:
{'owner': 'tester', 'scan_id': 123, 'beamline_id': 'test', 'uid': 'b2047c5c-65e8-4b17-a041-90c8c224e54f', 'time': 1432995057.724605}
set: (mover: motor), ({'pos': 5},), {}
   ret: None
trigger: (mover: motor), (), {}
   ret: None
read: (mover: motor), (), {}
   ret: {'pos': {'value': 5, 'timestamp': 1432995058.026932}}
Emitted RunStop:
{'time': 1432995058.233652, 'reason': '', 'exit_status': 'success', 'run_start': 'b2047c5c-65e8-4b17-a041-90c8c224e54f'}


Moving a motor and reading it back is boring. Let's add a detector.

In [8]:
def simple_scan2(motor, det):
    "Set, trigger motor, trigger detector, read"
    yield Msg('set', motor, {'pos': 5})
    yield Msg('trigger', motor)
    yield Msg('trigger', det)
    yield Msg('read', det)
    
RE.run(simple_scan2(motor, det))

Emitted RunStart:
{'owner': 'tester', 'scan_id': 123, 'beamline_id': 'test', 'uid': '3cc1189f-f7ef-4c73-8a41-4f21ee2a5b6e', 'time': 1432995064.493223}
set: (mover: motor), ({'pos': 5},), {}
   ret: None
trigger: (mover: motor), (), {}
   ret: None
trigger: (reader: sg), (), {}
   ret: None
read: (reader: sg), (), {}
   ret: {'intensity': {'timetamp': 1432995064.907875, 'value': 3.7266531720786709e-06}}
Emitted RunStop:
{'time': 1432995065.11085, 'reason': '', 'exit_status': 'success', 'run_start': '3cc1189f-f7ef-4c73-8a41-4f21ee2a5b6e'}


## There is two-way communication between the message generator and the Run Engine

Above we the three messages with the responses they generated from the RunEngine. We can use these responses to make our scan adaptive.

In [9]:
def adaptive_scan(motor, det, threshold):
    """Set, trigger, read until the detector reads intensity < threshold"""
    i = 0
    while True:
        print("LOOP %d" % i)
        yield Msg('set', motor, {'pos': i})
        yield Msg('trigger', motor)
        yield Msg('trigger', det)
        reading = yield Msg('read', det)
        if reading['intensity']['value'] < threshold:
            print('DONE')
            break
        i += 1

RE.run(adaptive_scan(motor, det, 0.2))

Emitted RunStart:
{'owner': 'tester', 'scan_id': 123, 'beamline_id': 'test', 'uid': 'cb21e9b6-0343-422c-817d-6859a18d9c6d', 'time': 1432995072.243196}
LOOP 0
set: (mover: motor), ({'pos': 0},), {}
   ret: None
trigger: (mover: motor), (), {}
   ret: None
trigger: (reader: sg), (), {}
   ret: None
read: (reader: sg), (), {}
   ret: {'intensity': {'timetamp': 1432995072.655072, 'value': 1.0}}
LOOP 1
set: (mover: motor), ({'pos': 1},), {}
   ret: None
trigger: (mover: motor), (), {}
   ret: None
trigger: (reader: sg), (), {}
   ret: None
read: (reader: sg), (), {}
   ret: {'intensity': {'timetamp': 1432995073.169031, 'value': 0.60653065971263342}}
LOOP 2
set: (mover: motor), ({'pos': 2},), {}
   ret: None
trigger: (mover: motor), (), {}
   ret: None
trigger: (reader: sg), (), {}
   ret: None
read: (reader: sg), (), {}
   ret: {'intensity': {'timetamp': 1432995073.690777, 'value': 0.1353352832366127}}
DONE
Emitted RunStop:
{'time': 1432995073.895493, 'reason': '', 'exit_status': 'success',

## Control timing with 'sleep' and 'wait'

The 'sleep' command is as simple as it sounds.

In [10]:
def sleepy_scan(motor, det):
    "Set, trigger motor, sleep for a fixed time, trigger detector, read"
    yield Msg('set', motor, {'pos': 5})
    yield Msg('trigger', motor)
    yield Msg('sleep', None, 2)  # units: seconds
    yield Msg('trigger', det)
    yield Msg('read', det)
    
RE.run(sleepy_scan(motor, det))

Emitted RunStart:
{'owner': 'tester', 'scan_id': 123, 'beamline_id': 'test', 'uid': '7040be31-773a-4170-be40-274d4cff7366', 'time': 1432995082.954654}
set: (mover: motor), ({'pos': 5},), {}
   ret: None
trigger: (mover: motor), (), {}
   ret: None
sleep: (None), (2,), {}
   ret: None
trigger: (reader: sg), (), {}
   ret: None
read: (reader: sg), (), {}
   ret: {'intensity': {'timetamp': 1432995085.475303, 'value': 3.7266531720786709e-06}}
Emitted RunStop:
{'time': 1432995085.683052, 'reason': '', 'exit_status': 'success', 'run_start': '7040be31-773a-4170-be40-274d4cff7366'}


The 'wait' command is more powerful. It watches for Movers (e.g., `motor`) to report being done.

### Wait for one motor to be done moving

In [3]:
def wait_one(motor, det):
    "Set, trigger, read"
    yield Msg('set', motor, {'pos': 5})
    yield Msg('trigger', motor, block_group='A')  # Add motor to group 'A'.
    yield Msg('wait', None, 'A')  # Wait for everything in group 'A' to report done.
    yield Msg('trigger', det)
    yield Msg('read', det)
    
RE.run(wait_one(motor, det))

Emitted RunStart:
{'scan_id': 123, 'uid': '6ec6a162-0a9f-4500-9b1f-200e2a9be499', 'owner': 'tester', 'time': 1432995419.37463, 'beamline_id': 'test'}
set: (mover: motor), ({'pos': 5},), {}
   ret: None
trigger: (mover: motor), (), {'block_group': 'A'}
   ret: None
wait: (None), ('A',), {}
   ret: {mover: motor}
trigger: (reader: sg), (), {}
   ret: None
read: (reader: sg), (), {}
   ret: {'intensity': {'timetamp': 1432995419.890234, 'value': 3.7266531720786709e-06}}
Emitted RunStop:
{'reason': '', 'run_start': '6ec6a162-0a9f-4500-9b1f-200e2a9be499', 'exit_status': 'success', 'time': 1432995420.096022}


Notice, in the log, that the response to `wait` is the set of Movers the scan was waiting on.

### Wait for two motors to both be done moving

In [4]:
def wait_multiple(motors, det):
    "Set motors, trigger all motors, wait for all motors to move."
    for motor in motors:
        yield Msg('set', motor, {'pos': 5})
        yield Msg('trigger', motor, block_group='A')  # Trigger each motor and add it to group 'A'.
    yield Msg('wait', None, 'A')  # Wait for everything in group 'A' to report done.
    yield Msg('trigger', det)
    yield Msg('read', det)

motor1 = Mover('motor1', ['pos'])
motor2 = Mover('motor2', ['pos'])

RE.run(wait_multiple([motor1, motor2], det))

Emitted RunStart:
{'scan_id': 123, 'uid': '52de6e9d-c762-4197-a5c2-6bd203a0c0c1', 'owner': 'tester', 'time': 1432995426.953379, 'beamline_id': 'test'}
set: (mover: motor1), ({'pos': 5},), {}
   ret: None
trigger: (mover: motor1), (), {'block_group': 'A'}
   ret: None
set: (mover: motor2), ({'pos': 5},), {}
   ret: None
trigger: (mover: motor2), (), {'block_group': 'A'}
   ret: None
wait: (None), ('A',), {}
   ret: {mover: motor2, mover: motor1}
trigger: (reader: sg), (), {}
   ret: None
read: (reader: sg), (), {}
   ret: {'intensity': {'timetamp': 1432995427.771697, 'value': 3.7266531720786709e-06}}
Emitted RunStop:
{'reason': '', 'run_start': '52de6e9d-c762-4197-a5c2-6bd203a0c0c1', 'exit_status': 'success', 'time': 1432995427.976239}


### Advanced Example: Wait for different groups of motors at different points in the run

If the `'A'` bit seems pointless, the payoff is here. We trigger all the motors at once, wait for the first two, read, wait for the last one, and read again. This is merely meant to show that complex control flow is possible.

In [5]:
def wait_complex(motors, det):
    "Set motors, trigger motors, wait for all motors to move."
    # Same as above...
    for motor in motors[:-1]:
        yield Msg('set', motor, {'pos': 5})
        yield Msg('trigger', motor, block_group='A')
        
    # ...but put the last motor is separate group.
    yield Msg('set', motors[-1], {'pos': 5})
    yield Msg('trigger', motors[-1], block_group='B')
    
    yield Msg('wait', None, 'A')  # Wait for everything in group 'A' to report done.
    yield Msg('trigger', det)
    yield Msg('read', det)
    
    yield Msg('wait', None, 'B')  # Wait for everything in group 'B' to report done.
    yield Msg('trigger', det)
    yield Msg('read', det)
    
motor3 = Mover('motor3', ['pos'])

RE.run(wait_complex([motor1, motor2, motor3], det))

Emitted RunStart:
{'scan_id': 123, 'uid': 'c89d40aa-166b-473c-aa62-6cbb158350e3', 'owner': 'tester', 'time': 1432995430.894365, 'beamline_id': 'test'}
set: (mover: motor1), ({'pos': 5},), {}
   ret: None
trigger: (mover: motor1), (), {'block_group': 'A'}
   ret: None
set: (mover: motor2), ({'pos': 5},), {}
   ret: None
trigger: (mover: motor2), (), {'block_group': 'A'}
   ret: None
set: (mover: motor3), ({'pos': 5},), {}
   ret: None
trigger: (mover: motor3), (), {'block_group': 'B'}
   ret: None
wait: (None), ('A',), {}
   ret: {mover: motor2, mover: motor1}
trigger: (reader: sg), (), {}
   ret: None
read: (reader: sg), (), {}
   ret: {'intensity': {'timetamp': 1432995432.017687, 'value': 3.7266531720786709e-06}}
wait: (None), ('B',), {}
   ret: {mover: motor3}
trigger: (reader: sg), (), {}
   ret: None
read: (reader: sg), (), {}
   ret: {'intensity': {'timetamp': 1432995432.324914, 'value': 3.7266531720786709e-06}}
Emitted RunStop:
{'reason': '', 'run_start': 'c89d40aa-166b-473c-aa62

## Runs can be aborted

### SIGINT (Ctrl+C) is reliably caught before each message is processed, even across threads.

The output below is truncated because it caught Ctrl+C (or, in the notebook, "Interrupt Kerenel").

In [17]:
RE.run(simple_scan(motor))

Emitted RunStart:
{'owner': 'tester', 'uid': '4df1da7d-f5ec-4d24-ac67-6312cc939b1a', 'scan_id': 123, 'time': 1432990069.884299, 'beamline_id': 'test'}
set: (mover: motor), ({'pos': 5},), {}
   response: None
trigger: (mover: motor), (), {}
   response: None


### Threading is optional -- switch it off for easier debugging

Again, we'll interrupt the scan. We get exactly the same result, but this time we see a full Traceback.

In [18]:
RE.run(simple_scan(motor), use_threading=False)

Emitted RunStart:
{'owner': 'tester', 'uid': '9149aac3-f2e8-456c-83d3-f8d86162c736', 'scan_id': 123, 'time': 1432990193.53064, 'beamline_id': 'test'}
set: (mover: motor), ({'pos': 5},), {}
   response: None
Emitted RunStop:
{'exit_status': 'abort', 'reason': '', 'run_start': '9149aac3-f2e8-456c-83d3-f8d86162c736', 'time': 1432990193.697828}


RunInterrupt: RunEngine detected a SIGINT (Ctrl+C) and aborted the scan. Records were created, but the run was marked with exit_status='abort'.

## Data can be consumed by Metadatastore and User Functions

In the examples above, the runs have been emitting RunStart and RunStop Documents, but no Events or Event Descriptors. We will add those now.

### Emitting Events and Event Descriptors

In [18]:
def simple_scan_saving(motor):
    "Set, trigger, read"
    yield Msg('create')
    yield Msg('set', motor, {'pos': 5})
    yield Msg('trigger', motor)
    yield Msg('read', motor)
    yield Msg('save')
    
RE.run(simple_scan_saving(motor))

Emitted RunStart:
{'owner': 'tester', 'scan_id': 123, 'beamline_id': 'test', 'uid': 'd6efa520-b57e-4ea4-abb6-7164a3839de2', 'time': 1432995211.20246}
create: (None), (), {}
   ret: None
set: (mover: motor), ({'pos': 5},), {}
   ret: None
trigger: (mover: motor), (), {}
   ret: None
read: (mover: motor), (), {}
   ret: {'pos': {'value': 5, 'timestamp': 1432995211.61009}}
Emitted Event Descriptor:
{'time': 1432995211.811159, 'uid': '96849beb-e998-4ff1-b184-290515078630', 'data_keys': {'pos': {'dtype': 'number', 'source': 'motor'}}, 'run_start': 'd6efa520-b57e-4ea4-abb6-7164a3839de2'}
Emitted Event:
{'seq_num': 1, 'data': {'pos': {'value': 5, 'timestamp': 1432995211.61009}}, 'descriptor': '96849beb-e998-4ff1-b184-290515078630', 'time': 1432995211.811301, 'uid': '31297c20-dd81-42b6-bed2-0550d2cf7215'}
save: (None), (), {}
   ret: None
Emitted RunStop:
{'time': 1432995211.916451, 'reason': '', 'exit_status': 'success', 'run_start': 'd6efa520-b57e-4ea4-abb6-7164a3839de2'}


The `'create'` and `'save'` commands collect all the reads between them into one Event. If that particular set of objects has never been bundled into an Event during this run, then an Event Descriptor is also created.

### Consuming Documents for Live Visualization and Analysis

In [ ]:
%run register_mds.py

register_mds(RE)